# Creating the web scraper

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import sklearn.metrics as sm


import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import csv
import requests
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup


from neo4j import GraphDatabase 
from py2neo import Graph,Node,Relationship

# Getting the dataset

In [3]:
data = pd.read_csv('charts.csv')

In [23]:
refined_data = data.loc[(data["rank"] == 1)]

In [24]:
new_index = [1,2,3,4,5,6,7,8,9,10,11,12,133,14,15,16,17,18,19,20]

In [25]:
new_data = refined_data.reset_index(drop = True)

In [26]:
refined_data.shape

(227253, 9)

In [27]:
index_names = new_data[new_data['trend'] == 'SAME_POSITION' ].index

Int64Index([     0,      4,      5,      6,      7,      8,      9,     10,
                11,     12,
            ...
            227242, 227243, 227244, 227245, 227246, 227247, 227249, 227250,
            227251, 227252],
           dtype='int64', length=193123)

In [28]:
new_data.drop(index_names, inplace=True)

In [29]:
new_data.shape

(34130, 9)

In [30]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0
25,Rockabye (feat. Sean Paul & Anne-Marie),1,2017-01-01,Clean Bandit,https://open.spotify.com/track/5knuzwU65gJK7IF...,Hungary,top200,MOVE_UP,6263.0
...,...,...,...,...,...,...,...,...,...
227143,Diamond in the rough,1,2021-07-30,Arik Ancelin,https://open.spotify.com/track/54AtiA08i3sDfJY...,Iceland,viral50,MOVE_UP,NaN
227144,Aadmi Chutiya Hai,1,2021-07-30,Rahgir,https://open.spotify.com/track/2PEJKORCdmceTW2...,India,viral50,MOVE_UP,NaN
227196,Bezos I,1,2021-07-31,Bo Burnham,https://open.spotify.com/track/0Kdj7nwaYQmvhxn...,Czech Republic,viral50,MOVE_UP,NaN
227202,Paar tundi veel,1,2021-07-31,Shanon,https://open.spotify.com/track/3LM9EDiw6CtKhCt...,Estonia,viral50,MOVE_UP,NaN


In [31]:
contain_values = new_data[new_data.artist.str.contains('Shakira')]
print (contain_values)

                          title  rank        date                    artist  \
98      Chantaje (feat. Maluma)     1  2017-01-02                   Shakira   
281     Chantaje (feat. Maluma)     1  2017-01-03                   Shakira   
583     Chantaje (feat. Maluma)     1  2017-01-04                   Shakira   
3133    Chantaje (feat. Maluma)     1  2017-01-07                   Shakira   
6260    Chantaje (feat. Maluma)     1  2017-01-12                   Shakira   
6779    Chantaje (feat. Maluma)     1  2017-01-14                   Shakira   
7657    Chantaje (feat. Maluma)     1  2017-01-15                   Shakira   
83302               Clandestino     1  2018-07-04           Shakira, Maluma   
83336               Clandestino     1  2018-07-04           Shakira, Maluma   
83714               Clandestino     1  2018-07-04           Shakira, Maluma   
84110               Clandestino     1  2018-07-05           Shakira, Maluma   
84111               Clandestino     1  2018-07-05   

In [13]:
#200403  I Feel It Coming
#209094   Save Your Tears
# 1 Starboy

## Reducing the amount of data

In [32]:
new_data.head()

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0
25,Rockabye (feat. Sean Paul & Anne-Marie),1,2017-01-01,Clean Bandit,https://open.spotify.com/track/5knuzwU65gJK7IF...,Hungary,top200,MOVE_UP,6263.0


In [15]:
#new_data = new_data.drop(new_data[new_data.index > 20].index)                          # Using logical condition
   

In [33]:
new_data.shape

(34130, 9)

In [17]:
new_data.iloc[140087:140100,:]

,title,rank,date,artist,url,region,chart,trend,streams


In [18]:
new_data

,title,rank,date,artist,url,region,chart,trend,streams
1,Starboy,1,2017-01-01,"The Weeknd, Daft Punk",https://open.spotify.com/track/5aAx2yezTd8zXrk...,Australia,top200,MOVE_UP,107350.0
2,Alone,1,2017-01-01,Alan Walker,https://open.spotify.com/track/0JiVRyTJcJnmlwC...,Austria,top200,MOVE_UP,12990.0
3,Hey Baby,1,2017-01-01,"Dimitri Vegas & Like Mike, Diplo, Deb’s Daughter",https://open.spotify.com/track/6yc377A8qxHIjPU...,Belgium,top200,MOVE_UP,23277.0
17,Hukutaan,1,2017-01-01,Arttu Lindeman,https://open.spotify.com/track/0i8qXP5jTcu3dPf...,Finland,top200,MOVE_UP,60220.0


# Import dataset

In [94]:
data = pd.read_csv('../data/music-subset.csv', index_col=0)

In [229]:
data.head(20)

,title,rank,date,artist,url,region,chart,trend,streams,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
index,,,,,,,,,,,,,,,,,,,,,
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,0.852,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.907,102.034,195840.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,0.663,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.533,99.935,259196.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,0.761,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.710,93.974,222560.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,0.508,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.555,180.044,205600.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,0.899,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.873,88.007,234320.0
5,Traicionera,6,2017-01-01,Sebastian Yatra,https://open.spotify.com/track/5J1c3M4EldCfNxX...,Argentina,top200,MOVE_DOWN,151140.0,0.776,...,11.0,-4.933,1.0,0.0638,0.14200,0.000000,0.2190,0.661,91.012,228467.0
6,Cuando Se Pone a Bailar,7,2017-01-01,Rombai,https://open.spotify.com/track/1MpKZi1zTXpERKw...,Argentina,top200,MOVE_DOWN,148369.0,0.588,...,11.0,-7.169,0.0,0.1730,0.08510,0.000027,0.0840,0.937,205.643,195274.0
7,Otra vez (feat. J Balvin),8,2017-01-01,Zion & Lennox,https://open.spotify.com/track/3QwBODjSEzelZyV...,Argentina,top200,MOVE_DOWN,143004.0,0.832,...,10.0,-5.429,1.0,0.1000,0.05590,0.000486,0.4400,0.704,96.016,209453.0
8,La Bicicleta,9,2017-01-01,"Carlos Vives, Shakira",https://open.spotify.com/track/0sXvAOmXgjR2QUq...,Argentina,top200,MOVE_UP,126389.0,0.736,...,0.0,-2.147,1.0,0.1290,0.19800,0.000002,0.3360,0.953,179.935,227707.0


# Extracting artist and track name

In [282]:
track_nr = 8

In [256]:
import re

def get_track_name(nr):
    

    track_string = str(data.title[nr])
    
    x = track_string.find(" (")
    y = track_string.find(" -")


    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string



def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [283]:
get_track_name(track_nr)

'La Bicicleta'

In [284]:
get_track_artist(track_nr)

'Carlos Vives'

# Getting URL

In [274]:
import re

def get_url(track_nr):
    
    track = str(get_track_name(track_nr))
    artist = str(get_track_artist(track_nr))
    
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', track)
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', artist)
    
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
        
    
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)
    

In [285]:
get_url(track_nr)

'https://genius.com/Carlos-Vives-La-Bicicleta-lyrics'

# Scraping credits

In [105]:
def extract_credits(URL):
    
    # "https://genius.com/Shakia-chantaje-lyrics" <- url to check for raised exception
    
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):


        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        s = soup.find('div', class_='SongInfo__Columns-nekw6x-2 lgBflw')


        featur = []
        writer = []
        label = []
        producer = []

        for line in s:
            div = line.find('div').text
            an = line.find_all('a')


            if 'Featuring' in div: 
                extract = [x.text for x in an]
                for x in extract:
                    featur.append(x)



            if 'Written By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    writer.append(x)



            if 'Label' in div:
                extract = [x.text for x in an]
                for x in extract:
                    label.append(x)

            if 'Produced By' in div:
                extract = [x.text for x in an]
                for x in extract:
                    producer.append(x)

    else:
        raise Exception("Sorry, no numbers below zero") 

    collection = {"Feature":featur, "Writer":writer, "Label":label, "Producer":producer}        
        
        
    return collection

In [286]:
#extLis = extract_credits(get_url(218900))
#extLis = extract_credits(get_url(200403))
#extLis = extract_credits(get_url(98))
extLis = extract_credits(get_url(track_nr))

# Dict to csv

In [287]:
credit_list = pd.DataFrame.from_dict(extLis, orient='index').T

### Path to neo4j import folder
Nicholas Stationær: C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv

In [288]:
df = pd.DataFrame.from_dict(credit_list) 
df.to_csv (r'C:\Users\Nmtur\.Neo4jDesktop\relate-data\dbmss\dbms-20b18527-223b-4ada-8e62-429ad982fc9b\import\collaborators.csv', index = False, header=True)

In [289]:
df

,Feature,Writer,Label,Producer
0,Shakira,Shakira,None,Andrés Castro
1,None,Andrés Castro,None,Carlos Vives
2,None,Carlos Vives,None,None


***

# Neo4j

In [110]:
# Create connection to database

class neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
                print("Failed to create the driver: ", e)
                
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed!", e)
        finally:
            if session is not None:
                session.close()
        return response
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

In [111]:
# Connecting to database
conn = neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="test123")


In [56]:
conn.query('''MATCH (n) RETURN n;''')

[]

In [290]:
artist = get_track_artist(track_nr)
track = get_track_name(track_nr)
track

'La Bicicleta'

In [291]:
artist

'Carlos Vives'

### Query to insert into neo4j

In [296]:
def run_querys(a, t):
    
    artist = a
    track = t
    
    conn.query('''CALL apoc.load.csv('collaborators.csv')
    YIELD lineNo, map, list
    
    WHERE map.Feature IS NOT NULL
    MERGE (f:Person {name:map.Feature})
    MERGE (a:Person {name:"'''+ artist + '''"})
    MERGE (w:Person {name:map.Writer})
    MERGE (p:Person {name:map.Producer})
    MERGE (l:Label {name:map.Label})
    MERGE (s:Song {name:"'''+ track +'''"})
    
    
    MERGE (f)-[:COLLABORATED]-(a)
    MERGE (f)-[:FEATURED]->(s)
    MERGE (a)-[:SANG]->(s)
    MERGE (w)-[:WROTE]->(s)
    MERGE (l)-[:RELEASED]->(s)
    MERGE (p)-[:PRODUCED]->(s);
    ''')
    
    conn.query('''MATCH (f:Person) WHERE (f.name = "") detach delete f;''')
    
    conn.query('''MATCH (l:Label) WHERE (l.name = "") detach delete l;''')
    
    conn.close()

In [293]:
run_querys(artist, track)

In [267]:
df

,Feature,Writer,Label,Producer
0,None,Nicky Jam,Sony Music Entertainment Colombia S.A,Chan El Genio
1,None,Kevin ADG,Sony Music Latin,Kevin ADG
2,None,Chan El Genio,Diego Abaroa,None
3,None,Bull Nene,None,None
4,None,Maluma,None,None
